## preprocess

In [1]:
import pandas as pd

In [2]:
import pymongo as mg
import pandas as pd
client = mg.MongoClient(host='mongodb://localhost:27017')

In [3]:
database = client['db_NHIS']

In [4]:
collection = database['Join Collection NSC2_BND_M20']

In [5]:
cursor = collection.find({})

In [6]:
list_NBM = list(cursor)

In [7]:
df_NBM = pd.DataFrame(list_NBM)
df_NBM[:2]

,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,...,OPRTN_YN,MDCARE_DD_CNT,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY
0,64e6cbf07574768bae8e9ed1,294364,1921LE,200707.0,T08-T14,W00-W19,2002080298703,26452,20020830,3,...,0,1,1,3,5.0,0.15,10590,3000,7590,2002
1,64e6cbf07574768bae8e9ed2,294364,1921LE,200707.0,T08-T14,W00-W19,2002090466395,63715,20020910,3,...,0,1,1,2,5.0,0.15,16520,4950,11570,2002


In [8]:
df_NBM['SICK_SYM1'].value_counts()

J209     8244
I10      4629
I109     3275
F_       3203
N_       3177
         ... 
H700        1
E058        1
M5397       1
T18         1
P289        1
Name: SICK_SYM1, Length: 3414, dtype: int64

In [9]:
len('F_'), len('M5397')

(2, 5)

In [10]:
def convertSICK_SYM(sick_sym):
    if len(sick_sym) < 3:
        return None
    else :
        return sick_sym[:3]

In [11]:
convertSICK_SYM('M5397')

'M53'

In [12]:
df_NBM['SICK_SYM1_3']= df_NBM['SICK_SYM1'].apply(convertSICK_SYM)

In [13]:
df_NBM['SICK_SYM1_3'].value_counts()

J20    8917
I10    8427
J03    4187
M54    3904
J06    3460
       ... 
G97       1
K01       1
S28       1
I72       1
P28       1
Name: SICK_SYM1_3, Length: 747, dtype: int64

In [14]:
df_NBM_drop=df_NBM.dropna()

In [15]:
df_NBM_drop.isnull().sum()

_id                   0
RN_INDI               0
BTH_YYYY              0
DTH_YYYYMM            0
COD1                  0
COD2                  0
RN_KEY                0
RN_INST               0
MDCARE_STRT_DT        0
FORM_CD               0
MCARE_SUBJ_CD         0
SICK_SYM1             0
OFIJ_TYPE             0
OPRTN_YN              0
MDCARE_DD_CNT         0
VSHSP_DD_CNT          0
TOT_PRSC_DD_CNT       0
MCARE_RSLT_TYPE       0
EDC_ADD_RT            0
ED_RC_TOT_AMT         0
EDC_SBA               0
EDC_INSUR_BRDN_AMT    0
STD_YYYY              0
SICK_SYM1_3           0
dtype: int64

In [16]:
df_NBM_drop['SICK_SYM1_3'].value_counts()

J20    68
E11    56
I10    50
J30    35
I42    32
       ..
R81     1
H36     1
K12     1
E03     1
H00     1
Name: SICK_SYM1_3, Length: 141, dtype: int64

In [17]:
target_list = df_NBM_drop['SICK_SYM1_3'].value_counts().index.to_list()

In [18]:
target_list.index('I10'), target_list.index('L02')

(2, 83)

In [19]:
def applyIndexNumber(sick_sym_3):
    indexNumber= target_list.index(sick_sym_3)
    return indexNumber

In [20]:
df_NBM_drop['SICK_SYM1_3_list']=df_NBM_drop['SICK_SYM1_3'].apply(applyIndexNumber)

C:\Users\5-20\AppData\Local\Temp\ipykernel_8412\158970229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_NBM_drop['SICK_SYM1_3_list']=df_NBM_drop['SICK_SYM1_3'].apply(applyIndexNumber)


In [21]:
df_NBM_drop[:2]

,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,...,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY,SICK_SYM1_3,SICK_SYM1_3_list
0,64e6cbf07574768bae8e9ed1,294364,1921LE,200707.0,T08-T14,W00-W19,2002080298703,26452,20020830,3,...,1,3,5.0,0.15,10590,3000,7590,2002,L02,83
3,64e6cbf07574768bae8e9ed4,294364,1921LE,200707.0,T08-T14,W00-W19,2002020488631,23506,20020208,3,...,1,3,5.0,0.15,12920,3000,9920,2002,K29,8


In [22]:
target =df_NBM_drop['SICK_SYM1_3_list']
features = df_NBM_drop[['EDC_ADD_RT', 'ED_RC_TOT_AMT']]
target.shape, features.shape

((829,), (829, 2))

In [23]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=111)
features_train.shape, target_train.shape, features_test.shape, target_test.shape

((621, 2), (621,), (208, 2), (208,))

### 모델학습

In [24]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

In [25]:
model = DecisionTreeClassifier() # 순서 : 모델만들기

In [26]:
from sklearn.model_selection import GridSearchCV 

In [37]:
# 3 x 2 x 3 x cv(3) =54개의 모델 구성됨. 
hyper_params = {'min_samples_leaf' : range(2,5) #3
               ,'max_depth':range(2,5) #2
               ,'min_samples_split':range(2,5)}#3 

### 평가 score : 분류 - 정확도, 예측 -R square

In [39]:
from sklearn.metrics import f1_score

In [46]:
grid_search = GridSearchCV(model, param_grid=hyper_params, cv=2,
                           verbose=1, scoring=f1_score) #학습된 내용을 출력해줘라. 

In [47]:
#54개중 가장 큰 것을 가져옴 /범주형 평가 수치 :정확도(accurance)가 높은 것이 최고!
#fit을 해서 내부에 54개 데이터를 가지고 있음 
grid_search.fit(features, target)

Fitting 2 folds for each of 27 candidates, totalling 54 fits


C:\Users\5-20\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
C:\Users\5-20\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\5-20\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: f1_score() takes 2 positional arguments but 3 were given

  warnings.warn(
C:\Users\5-20\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\5-20\anaconda3\Lib\

GridSearchCV(cv=2, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(2, 5),
                         'min_samples_leaf': range(2, 5),
                         'min_samples_split': range(2, 5)},
             scoring=<function f1_score at 0x0000018E31EC3E20>, verbose=1)

In [48]:
grid_search.best_estimator_

DecisionTreeClassifier(max_depth=2, min_samples_leaf=2)

In [49]:
#nan =  0.1이라서, 찾기 힘들다. 
grid_search.best_score_, grid_search.best_params_

(nan, {'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 2})

In [50]:
#최고의 성능을 가진 모델만 담게됨. 
best_model = grid_search.best_estimator_
best_model

DecisionTreeClassifier(max_depth=2, min_samples_leaf=2)

In [33]:
target_test_predict = best_model.predict(features_test)
target_test_predict # best_model 찍으면 값이 나옴. (그 중 최고인 것들만 찍어냄) 

array([18,  2,  0,  1,  2,  2, 13,  0,  2,  3,  0,  9,  0,  1,  9, 34,  0,
        8, 55,  0,  4, 14,  6,  7,  9,  2, 34,  9,  2,  6,  2,  0,  5,  0,
        0,  1,  9,  3,  9,  0, 10,  1, 42, 10, 13,  0, 20,  1,  1,  0,  1,
        7, 11,  0, 34,  0, 34, 21, 10,  1,  1, 55,  0,  9,  0, 12, 55,  3,
       12,  1, 11,  1,  4, 10, 10, 20,  2,  1,  1,  6, 55,  0, 11,  1, 12,
        5, 20,  0,  3,  1,  0,  1,  0,  1, 11,  0,  9,  0,  1,  1,  6,  0,
       21,  7, 25,  6,  3,  4, 15,  4,  7, 12,  5,  0,  0, 21, 15,  2, 34,
        7,  0,  4, 55,  9,  2,  0,  0,  2, 20,  5,  5,  2,  0, 11,  0,  0,
       25,  0, 18,  6,  6,  4,  2,  0,  2,  1,  0,  6,  0,  9,  6, 14,  1,
        2, 30, 12,  5,  1,  2, 10,  4, 18,  2,  0,  0,  4, 14,  6,  3,  2,
        5, 30,  1,  0,  0,  0,  0,  2, 18,  4,  0, 14, 20,  1,  2,  5, 18,
        0,  4,  3,  2,  1,  1,  0,  4,  1,  1, 12,  1,  0,  5,  5,  6,  0,
        4, 34,  1, 10], dtype=int64)

In [34]:
from sklearn.metrics import classification_report

In [36]:
print(classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           0       0.09      0.27      0.13        15
           1       0.07      0.29      0.11         7
           2       0.43      0.45      0.44        20
           3       0.00      0.00      0.00         6
           4       0.42      0.62      0.50         8
           5       0.10      0.25      0.14         4
           6       0.00      0.00      0.00         4
           7       0.60      0.50      0.55         6
           8       1.00      0.10      0.18        10
           9       0.10      0.20      0.13         5
          10       0.14      0.20      0.17         5
          11       0.00      0.00      0.00         2
          12       0.67      1.00      0.80         4
          13       0.50      0.50      0.50         2
          14       0.50      0.50      0.50         4
          15       0.00      0.00      0.00         3
          16       0.00      0.00      0.00         2
          17       0.00    

C:\Users\5-20\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\5-20\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\5-20\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\5-20\anaconda3\Lib\site-packages\sklearn\metrics\_classif